In [1]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: C:\Python312\python.exe -m pip install --upgrade pip


In [2]:
!pip install torch 

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: C:\Python312\python.exe -m pip install --upgrade pip


In [3]:
!pip install scipy

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: C:\Python312\python.exe -m pip install --upgrade pip


In [4]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: C:\Python312\python.exe -m pip install --upgrade pip


In [5]:
!pip install tqdm

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: C:\Python312\python.exe -m pip install --upgrade pip


In [6]:
# ==========================
# 📦 Imports
# ==========================
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import pandas as pd
from tqdm import tqdm

# ==========================
# ⚙️ Setup
# ==========================
device = "cuda" if torch.cuda.is_available() else "cpu"
MODEL = "cardiffnlp/twitter-roberta-base-sentiment"

tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL).to(device)
model.eval()

# ==========================
# 💬 Improved Function: Better Neutral Detection
# ==========================
def sentiment_1_to_5_improved(text: str):
    """
    Compute sentiment score on a 1-5 scale with better neutral detection.
    
    Uses multiple strategies:
    1. Weighted average for score
    2. Confidence thresholds for classification
    3. Neutral zone detection
    """
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)

    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits[0].cpu().numpy()
    probs = softmax(logits)  # [neg, neu, pos]

    p_neg, p_neu, p_pos = probs
    
    # Weighted average → 1-5 scale
    score = 1 * p_neg + 3 * p_neu + 5 * p_pos
    
    # Improved classification logic
    # Strategy 1: High neutral probability (> 0.5)
    if p_neu > 0.5:
        label = "neutral"
    # Strategy 2: Balanced negative and positive (both > 0.2)
    elif p_neg > 0.2 and p_pos > 0.2:
        label = "neutral"
    # Strategy 3: Score-based thresholds with tighter ranges
    elif score >= 3.8:
        label = "positive"
    elif score <= 2.2:
        label = "negative"
    else:
        # In the neutral zone (2.2 - 3.8)
        # Check confidence: if no probability dominates, it's neutral
        max_prob = max(p_neg, p_neu, p_pos)
        if max_prob < 0.5:  # Low confidence
            label = "neutral"
        elif p_neu > p_neg and p_neu > p_pos:
            label = "neutral"
        elif score > 3.0:
            label = "slightly positive"
        else:
            label = "slightly negative"
    
    # Calculate confidence
    confidence = max(p_neg, p_neu, p_pos)
    
    return {
        "score": round(float(score), 2),
        "label": label,
        "confidence": round(float(confidence), 3),
        "probabilities": {
            "neg": round(float(p_neg), 3),
            "neu": round(float(p_neu), 3),
            "pos": round(float(p_pos), 3)
        }
    }

# ==========================
# 🧪 Extended Examples with More Neutral Cases
# ==========================
example_texts = [
    # Clearly Positive
    "T-Mobile's 5G is amazing! Super fast and reliable.",
    "I switched from Verizon to T-Mobile — best decision ever!",
    
    # Clearly Negative
    "The network keeps dropping today 😒",
    "Why is my service so bad lately??",
    "Terrible customer service. Never using them again!",
    
    # Truly Neutral
    "Customer support was okay, nothing special.",
    "I called customer service today.",
    "T-Mobile is a wireless carrier.",
    "The store is open from 9 to 5.",
    
    # Mixed/Ambiguous
    "The price is good but the coverage is spotty.",
    "Great speeds but poor customer service.",
    "It works fine, I guess.",
    "Not bad, not great either.",
]

print("=" * 80)
print("SENTIMENT ANALYSIS RESULTS")
print("=" * 80)

for text in example_texts:
    result = sentiment_1_to_5_improved(text)
    
    # Color coding for terminal output
    label_color = {
        "positive": "✅",
        "slightly positive": "🟢",
        "neutral": "⚪",
        "slightly negative": "🟡",
        "negative": "❌"
    }
    
    icon = label_color.get(result["label"], "⚪")
    
    print(f"\n{icon} Text: {text}")
    print(f"   Score: {result['score']}/5.0")
    print(f"   Label: {result['label'].upper()}")
    print(f"   Confidence: {result['confidence']*100:.1f}%")
    print(f"   Probabilities: Neg={result['probabilities']['neg']} | "
          f"Neu={result['probabilities']['neu']} | Pos={result['probabilities']['pos']}")

# ==========================
# 📊 Batch Processing Function
# ==========================
def process_csv_with_improved_sentiment(input_file, output_file):
    """
    Process a CSV file with improved sentiment analysis.
    
    Args:
        input_file: Path to input CSV (must have 'text' column)
        output_file: Path to save results
    """
    print(f"\n📂 Loading {input_file}...")
    df = pd.read_csv(input_file)
    
    if 'text' not in df.columns:
        raise ValueError("CSV must have a 'text' column")
    
    print(f"📊 Processing {len(df)} texts...")
    
    # Apply sentiment analysis with progress bar
    tqdm.pandas(desc="Analyzing sentiment")
    results = df["text"].progress_apply(sentiment_1_to_5_improved)
    
    # Expand results into separate columns
    df["sentiment_score"] = results.apply(lambda x: x["score"])
    df["sentiment_label"] = results.apply(lambda x: x["label"])
    df["confidence"] = results.apply(lambda x: x["confidence"])
    df["prob_negative"] = results.apply(lambda x: x["probabilities"]["neg"])
    df["prob_neutral"] = results.apply(lambda x: x["probabilities"]["neu"])
    df["prob_positive"] = results.apply(lambda x: x["probabilities"]["pos"])
    
    # Save results
    df.to_csv(output_file, index=False)
    
    # Print summary statistics
    print(f"\n✅ Saved results to {output_file}")
    print(f"\n📈 Summary Statistics:")
    print(f"   Total texts: {len(df)}")
    print(f"   Sentiment distribution:")
    print(df["sentiment_label"].value_counts().to_string(prefix="      "))
    print(f"\n   Average score: {df['sentiment_score'].mean():.2f}")
    print(f"   Average confidence: {df['confidence'].mean():.2f}")

# ==========================
# 🎯 Usage Example
# ==========================
"""
# To process your own CSV file:
process_csv_with_improved_sentiment(
    "tmobile_tweets.csv", 
    "tmobile_sentiment_improved_results.csv"
)
"""

c:\Users\kbmlg\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\kbmlg\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kbmlg\.cache\huggingface\hub\models--cardiffnlp--twitter-roberta-base-sentiment. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to

SENTIMENT ANALYSIS RESULTS

✅ Text: T-Mobile's 5G is amazing! Super fast and reliable.
   Score: 4.98/5.0
   Label: POSITIVE
   Confidence: 99.1%
   Probabilities: Neg=0.002 | Neu=0.007 | Pos=0.991


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`



✅ Text: I switched from Verizon to T-Mobile — best decision ever!
   Score: 4.97/5.0
   Label: POSITIVE
   Confidence: 98.4%
   Probabilities: Neg=0.001 | Neu=0.015 | Pos=0.984

❌ Text: The network keeps dropping today 😒
   Score: 1.1/5.0
   Label: NEGATIVE
   Confidence: 95.5%
   Probabilities: Neg=0.955 | Neu=0.039 | Pos=0.006

❌ Text: Why is my service so bad lately??
   Score: 1.05/5.0
   Label: NEGATIVE
   Confidence: 97.6%
   Probabilities: Neg=0.976 | Neu=0.022 | Pos=0.002

❌ Text: Terrible customer service. Never using them again!
   Score: 1.05/5.0
   Label: NEGATIVE
   Confidence: 98.0%
   Probabilities: Neg=0.98 | Neu=0.017 | Pos=0.003

✅ Text: Customer support was okay, nothing special.
   Score: 4.08/5.0
   Label: POSITIVE
   Confidence: 60.4%
   Probabilities: Neg=0.064 | Neu=0.331 | Pos=0.604

⚪ Text: I called customer service today.
   Score: 2.73/5.0
   Label: NEUTRAL
   Confidence: 74.7%
   Probabilities: Neg=0.194 | Neu=0.747 | Pos=0.059

✅ Text: T-Mobile is a wirel

'\n# To process your own CSV file:\nprocess_csv_with_improved_sentiment(\n    "tmobile_tweets.csv", \n    "tmobile_sentiment_improved_results.csv"\n)\n'

In [7]:
import sys
!{sys.executable} -m pip install transformers --quiet


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import sys
!{sys.executable} -m pip install scipy --quiet


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import sys
!{sys.executable} -m pip install pandas --quiet


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
